In [2]:
from gates import *

In [10]:
Mux8Way16(a=(1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
          b=(0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0),
          c=(0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0),
          d=(0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0),
          e=(1,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0),
          f=(1,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0),
          g=(1,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0),
          h=(1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1),
          sel=(1,1,1))

(1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1)